In [1]:
import pandas as pd
import numpy as np


In [33]:
investigated_items_path = "/Volumes/GoogleDrive/.shortcut-targets-by-id/10oYqI9u7nCLK0q7xF2CvGGIQVokusjaI/Exjobb/7. Data collection/investigated_items.xlsx"
items = pd.read_excel(investigated_items_path,"Sheet1")
items.head(3)

,Interesting items
0,1030-61460
1,11033998
2,11033999


In [38]:
excel_path = "/Volumes/GoogleDrive/.shortcut-targets-by-id/10oYqI9u7nCLK0q7xF2CvGGIQVokusjaI/Exjobb/7. Data collection/item_demand_size_distributions.xlsx"
writer = pd.ExcelWriter(excel_path)

for item_code in items["Interesting items"]:
    demand_history_dir = "/Volumes/GoogleDrive/.shortcut-targets-by-id/10oYqI9u7nCLK0q7xF2CvGGIQVokusjaI/Exjobb/7. Data collection/item_demand_history_csv_files"
    demand_history_path = f"{demand_history_dir}/dh_{item_code}.csv"
    demand_history_df = pd.read_csv(demand_history_path)

    input_dir = "/Volumes/GoogleDrive/.shortcut-targets-by-id/10oYqI9u7nCLK0q7xF2CvGGIQVokusjaI/Exjobb/7. Data collection/item_input_csv_files"
    input_path = f"{input_dir}/item_{item_code}_input.csv"
    input_df = pd.read_csv(input_path)

    # Create output df
    demand_size_dist_df = pd.DataFrame()

    for name in input_df["Name"]:
        # Picking out demands the the dealer at hand.
        demand_history_dealer_df = demand_history_df[demand_history_df["Warehouse"] == name]

        # Removing demands of -1
        demand_history_dealer_df = demand_history_dealer_df.loc[demand_history_dealer_df["Demanded qty."] != -1]
        
        demand_count_dict = {}
        for q in demand_history_dealer_df["Demanded qty."]:
            if q in demand_count_dict:
                demand_count_dict[q] += 1
            else:
                demand_count_dict[q] = 1
        total_amount_of_orders = len(demand_history_dealer_df)

        demand_frequency_dict = {}
        for key in demand_count_dict.keys():
            demand_frequency_dict[key] = demand_count_dict[key]/total_amount_of_orders

        max_demand_size = max(demand_frequency_dict.keys())

        # Index 0 contains freq of demand size 1 and so on.
        demand_size_frequency_list = []
        for i in range(max_demand_size):
            k = i+1
            if k in demand_frequency_dict:
                demand_size_frequency_list.append(demand_frequency_dict[k])
            else:
                demand_size_frequency_list.append(0)

        demand_size_arr = pd.Series(demand_size_frequency_list)
        demand_size_dist_df[name] = demand_size_arr

    print(demand_size_dist_df)

    # Saving to csv
    csv_dir = "/Volumes/GoogleDrive/.shortcut-targets-by-id/10oYqI9u7nCLK0q7xF2CvGGIQVokusjaI/Exjobb/7. Data collection/item_demand_size_distributions_csv_files"
    csv_path = f"{csv_dir}/item_{item_code}_demand_size_dist"
    demand_size_dist_df.to_csv(csv_path)

    # Writing to excel
    excel_sheet_name = f"item_{item_code}_demand_size_dist"
    demand_size_dist_df.to_excel(writer, sheet_name = excel_sheet_name) 

writer.save()
    

   Bartlett  Bloemfontein  Capetown    Durban  George  Kimberley  Middelburg  \
0  0.839286      0.967742  0.924658  0.968750     1.0       0.84         1.0   
1  0.098214      0.032258  0.047945  0.020833     NaN       0.16         NaN   
2  0.017857           NaN  0.013699  0.000000     NaN        NaN         NaN   
3  0.000000           NaN  0.006849  0.000000     NaN        NaN         NaN   
4  0.000000           NaN       NaN  0.000000     NaN        NaN         NaN   
5  0.026786           NaN       NaN  0.000000     NaN        NaN         NaN   
6  0.000000           NaN       NaN  0.000000     NaN        NaN         NaN   
7  0.000000           NaN       NaN  0.000000     NaN        NaN         NaN   
8  0.008929           NaN       NaN  0.000000     NaN        NaN         NaN   
9  0.008929           NaN       NaN  0.010417     NaN        NaN         NaN   

   Nelspruit  Port Elizabeth  Richards Bay  Rustenburg  Steelpoort  \
0   0.794118        0.904762      0.642857       

/opt/anaconda3/envs/master_thesis/lib/python3.10/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


    Bartlett  Bloemfontein  Capetown    Durban  George  Kimberley  \
0   0.842857      0.859649  0.986301  0.981043     1.0   0.957143   
1   0.053333      0.087719  0.013699  0.018957     NaN   0.042857   
2   0.010476      0.035088       NaN       NaN     NaN        NaN   
3   0.013333      0.000000       NaN       NaN     NaN        NaN   
4   0.011429      0.017544       NaN       NaN     NaN        NaN   
5   0.002857           NaN       NaN       NaN     NaN        NaN   
6   0.000952           NaN       NaN       NaN     NaN        NaN   
7   0.006667           NaN       NaN       NaN     NaN        NaN   
8   0.000000           NaN       NaN       NaN     NaN        NaN   
9   0.006667           NaN       NaN       NaN     NaN        NaN   
10  0.000952           NaN       NaN       NaN     NaN        NaN   
11  0.009524           NaN       NaN       NaN     NaN        NaN   
12  0.000000           NaN       NaN       NaN     NaN        NaN   
13  0.000952           NaN       N

ValueError: max() arg is an empty sequence